# PART I: TITLE CREATION

In [1]:
import pandas as pd
import os
import numpy as np
import re
from collections import Counter
import spacy
from spacy.language import Language
from spacy_language_detection import LanguageDetector

In [2]:
#set globals

#list of all ports in vol
port_names = [
    "Newchwang",
    "Tientsin",
    "Chefoo",
    "Hankow",
    "Kiukiang",
    "Chinkiang",
    "Shanghai",
    "Ningpo",
    "Foochow",
    "Tamsui",
    "Takow",
    "Amoy",
    "Swatow",
    "Canton"
]

#list of all table types in vols
g_all_table_types = [
    "Tonnage",
    "Values",
    "Articles",
    "Revenue",
    "Population"
]

#source path
source_path = '/Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/'

#### Create local directory for combined csv and txt files

`cp ".../annual_trade_reports/csv/*" ".../annual_trade_reports/csv_and_txt"`
`cp ".../annual_trade_reports/txt/*" ".../annual_trade_reports/csv_and_txt"`

In [3]:
def title_maker(year):
    
    folder_path = source_path + str(year) + '/csv_and_txt'
    
    files_list = os.listdir(folder_path)
    
    #dataframe with names of all the files in the folder 
    df = pd.DataFrame(files_list,columns=['file_name'])
    
    df = df.sort_values(by="file_name",ascending=True).reset_index().drop(columns=['index'])
    
    #get file type (csv or txt)
    df["file_type"] = df["file_name"].str[-3:]
    
    df["file_name"] = df["file_name"].str[:-4]
    
    df["title_port"] = np.nan
    
    df['title_port'] = df['title_port'].astype(object)

    df["title_table_type"] = np.nan
    
    df['title_table_type'] = df['title_table_type'].astype(object)
    
    #list that will contain the page numbers of all content pages
    content_pages = []
    
    #loops through each txt file in the folder
    for index, row in df.iterrows():
        
        if row['file_type'] == 'txt':
            
            file_path = source_path + str(year) + '/txt/' + str(df.loc[index]['file_name'])+'.txt'
            file = open(file_path, 'r')
            Lines = file.readlines()
            
            #removes trailing characters from each line in the file
            lines_list = [line.strip('\n').strip('.').title() for line in Lines]
            
            #removes roman numerical characters from the beginning of all lines (this is for the port name titles)  
            combinations_to_remove = [r"I\.?[-—─-]", r"Ii\.?[-—─-]",r"Iii\.?[-—─-]",r"Iv\.?[-—─-]",r"V\.?[-—─-]",r"Vi\.?[-—─-]",r"Vii\.?[-—─-]",r"Viii\.?[-—─-]",r"Ix\.?[-—─-]"]
            pattern = '|'.join(combinations_to_remove)
            lines_list = [re.sub(pattern, "", line) for line in lines_list]
            
            #convert lists to sets
            file_set = set(lines_list)
            all_ports = set(port_names)
            all_table_types = set(g_all_table_types)
            
            #check whether any port names occur in the file
            common_elements_title_port = file_set.intersection(all_ports)

            # Check if exactly one port name appears
            if len(common_elements_title_port) == 1:
                #if yes, this file is the cover/starting page for the port whose name appears  
                df.at[index, 'title_port'] = list(common_elements_title_port)[0]

            #check if any table type titles occur in the file 
            common_elements_title_table = file_set.intersection(all_table_types)
            
            #check if this file contains the word "Contents" once 
            if len(file_set.intersection({"Contents"})) == 1:
                #if yes, this page is the contents page
                content_pages.append(row['file_name'])
            #else, check if exactly one table type name appears in this file 
            elif len(common_elements_title_table) == 1:
                #if yes, this is the cover/starting page for that table type
                df.at[index, 'title_table_type'] = list(common_elements_title_table)[0]
            #else, check if exactly two different table type names appear in this file 
            elif len(common_elements_title_table) == 2:
                #if yes, this is the cover/starting page for that table type
                df.at[index, 'title_table_type'] = ','.join(common_elements_title_table)
    
    #forward fills the title column, ie. every page takes the value of the last valid/occuring title port name  
    #df['title_port'] = df['title_port'].fillna(method='ffill')
    df['title_port'] = df['title_port'].ffill()
    
    df.loc[0]['title_port']=df.loc[1]['title_port']
    
    #within the same port name, every page takes the value of the last valid/occuring table type   
    #df["new_table_type"] = df.groupby('title_port')['title_table_type'].fillna(method="ffill")
    df["new_table_type"] = df.groupby('title_port')['title_table_type'].ffill().infer_objects(copy=False)
    
    #marks all the content pages
    df.loc[df['file_name'].isin(content_pages), 'new_table_type'] = "Contents"
    
    df = df.drop(columns={"title_table_type"})
    df = df.rename(columns={"new_table_type":"title_table_type"})
    
    df["year"] = year
    df["file_path"] = df["file_name"] +"."+ df["file_type"]
    
    #files that come before the table types are categorized as "Notes"   
    df = df.fillna("Notes")
    
    df["title"] = df["title_port"]+ "-" +df["title_table_type"]
    
    return df[["title_port","title_table_type","year","file_path","title"]]
    

In [4]:
df_statistics = title_maker(1873)

/var/folders/x5/8k4j_05j26vccc6wvb8y4cw00000gq/T/ipykernel_23224/141445153.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[0]['title_port']=df.loc[1]['title_port']


In [5]:
df_statistics

,title_port,title_table_type,year,file_path,title
0,Notes,Notes,1873,.DS_S.ore,Notes-Notes
1,Notes,Notes,1873,44319541.txt,Notes-Notes
2,Notes,Notes,1873,44319542.txt,Notes-Notes
3,Notes,Notes,1873,44319543.txt,Notes-Notes
4,Notes,Notes,1873,44319544.txt,Notes-Notes
...,...,...,...,...,...
754,Tientsin,Population,1873,44319950.txt,Tientsin-Population
755,Tientsin,Population,1873,44319950_a.csv,Tientsin-Population
756,Tientsin,Population,1873,44319950_b.csv,Tientsin-Population
757,Tientsin,Population,1873,44319951.txt,Tientsin-Population


# PART II: NLP ENTITIES CREATION

### II(a) Create entities for all csv files

In [6]:
# Define the language detector factory function
def get_lang_detector(nlp, name):
    return LanguageDetector(seed=42)

# Load the spaCy model and register the language detector factory if not already registered
nlp = spacy.load('en_core_web_sm')
if 'language_detector' not in nlp.pipe_names:
    Language.factory("language_detector", func=get_lang_detector)
    nlp.add_pipe('language_detector', last=True)

@Language.component("filter_non_english_entities")
def filter_non_english_entities(doc):
    filtered_ents = []
    for ent in doc.ents:
        if all(token.is_alpha for token in ent):
            filtered_ents.append(ent)
    doc.ents = filtered_ents
    return doc

# Add the custom filter component to remove non-English entities
if 'filter_non_english_entities' not in nlp.pipe_names:
    nlp.add_pipe("filter_non_english_entities", last=True)

In [ ]:
def nlp_entities_maker_csv_orig(year):
    folder_path_csv = source_path + str(year) + '/csv'
    files_list = os.listdir(folder_path_csv)
    csv_files_df = pd.DataFrame(files_list,columns = ['file_name'])
    csv_files_df['nlp_entities'] = '[]'
    
    for index, row in csv_files_df.iterrows():
    
        df = pd.read_csv(folder_path_csv + '/' +str(row["file_name"]))
        
        #method 1 works only on "Special" files that have names of specific goods 
        try:
            df["DESCRIPTION OF GOODS."] = df["DESCRIPTION OF GOODS."].fillna("").astype(str)
            
            # Convert the description of goods column into a single string
            combined_text = ' '.join(df["DESCRIPTION OF GOODS."].tolist())
    
            #NER model
            def get_lang_detector(nlp, name):
                return LanguageDetector(seed=42)
    
            nlp = spacy.load('en_core_web_sm')
            Language.factory("language_detector", func=get_lang_detector)
            nlp.add_pipe('language_detector', last=True)
    
            doc = nlp(combined_text)
    
            # Extract and filter named entities with the desired label ('WORK_OF_ART')
            entities = [ent.text for ent in doc.ents if ent.label_ == 'WORK_OF_ART']
    
            # Count the occurrences of each named entity
            entities_counter = Counter(entities)
    
            # Get the top 7 most important 'WORK_OF_ART' entities based on frequency
            top_7_work_of_art_entities = entities_counter.most_common(7)
    
            csv_files_df.at[index, 'nlp_entities'] = top_7_work_of_art_entities
        
        #method 2: works on rest of the csv files
        except KeyError: 
            #converts all columns to string and concatenate their values into a single string
            combined_text = ' '.join(df.astype(str).apply(lambda x: ' '.join(x), axis=1).tolist())
    
            #includs the column names as potential named entities
            column_names = list(df.columns)
            combined_text += ' '.join(column_names)
    
            #NER model
            def get_lang_detector(nlp, name):
                return LanguageDetector(seed=42)
                            
            nlp = spacy.load('en_core_web_sm')
            Language.factory("language_detector", func=get_lang_detector)
            nlp.add_pipe('language_detector', last=True)
    
            doc = nlp(combined_text)
    
            #excludes entities with digits and entities containing 'NaN'
            entities = [ent.text for ent in doc.ents if not any(char.isdigit() for char in ent.text) and 'nan' not in ent.text.lower()]
    
            #counts the occurrences of each entity
            entity_counts = Counter(entities)
    
            #the 7 most common entities
            most_common_entities = entity_counts.most_common(7)
    
            csv_files_df.at[index, 'nlp_entities'] = most_common_entities
            
            csv_files_df = csv_files_df.rename(columns={"file_name":"file_path"})
            
    return csv_files_df

In [45]:
def nlp_entities_maker_csv(year):
    folder_path_csv = source_path + str(year) + '/csv'
    files_list = os.listdir(folder_path_csv)
    csv_files_df = pd.DataFrame(files_list, columns=['file_name'])
    csv_files_df['nlp_entities'] = '[]'

    for index, row in csv_files_df.iterrows():
        file_path = os.path.join(folder_path_csv, row["file_name"])
        df = pd.read_csv(file_path)

        # Load the NER model
        nlp = spacy.load('en_core_web_sm')
        
        if 'DESCRIPTION OF GOODS.' in df.columns:
            df["DESCRIPTION OF GOODS."] = df["DESCRIPTION OF GOODS."].fillna("").astype(str)
            combined_text = ' '.join(df["DESCRIPTION OF GOODS."].tolist())
        else:
            combined_text = ' '.join(df.astype(str).apply(lambda x: ' '.join(x), axis=1).tolist())
            column_names = list(df.columns)
            combined_text += ' ' + ' '.join(column_names)

        # Check if 'language_detector' factory is already registered
        if 'language_detector' not in nlp.pipe_names:
            Language.factory("language_detector", func=get_lang_detector)
            nlp.add_pipe('language_detector', last=True)

        doc = nlp(combined_text)

        entities = [ent.text for ent in doc.ents if ent.label_ == 'WORK_OF_ART' or (not any(char.isdigit() for char in ent.text) and 'nan' not in ent.text.lower())]
        entity_counts = Counter(entities)
        most_common_entities = entity_counts.most_common(7)
        
        csv_files_df.at[index, 'nlp_entities'] = most_common_entities
    
    csv_files_df = csv_files_df.rename(columns={"file_name": "file_path"})
    return csv_files_df

### II(b) Create entities for all txt files

In [40]:
#clean all txt files first
def clean_txt_files(year):
    folder_path_txt = source_path + str(year) + '/txt'
    files_list = os.listdir(folder_path_txt)
    txt_files_df = pd.DataFrame(files_list,columns=['file_name'])
    
    for index, row in txt_files_df.iterrows():
        file_path = folder_path_txt + '/' + str(row['file_name'])
        with open(file_path, 'rb') as file:
            dirty_txt = file.read()
    
        clean_data = dirty_txt.decode('utf-8', 'ignore')

        # Write the clean data to the output file in text mode
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(clean_data)

In [41]:
clean_txt_files(1873)

In [42]:
def nlp_entities_maker_txt(year):
    
    # Load the NER model
    nlp = spacy.load('en_core_web_sm')
    
    # Check if 'language_detector' factory is already registered
    if 'language_detector' not in nlp.pipe_names:
        Language.factory("language_detector", func=get_lang_detector)
        nlp.add_pipe('language_detector', last=True)

    # Add the custom filter component to remove non-English entities
    nlp.add_pipe("filter_non_english_entities", last=True)

    folder_path_txt = source_path + str(year) + '/txt'
    files_list = os.listdir(folder_path_txt)
    txt_files_df = pd.DataFrame(files_list, columns=['file_name'])
    txt_files_df['nlp_entities'] = '[]'

    for index, row in txt_files_df.iterrows():
        file_path = folder_path_txt + '/' + str(row['file_name'])
        
        # Read the content of the text file
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        doc = nlp(text)

        # Excludes entities with digits and entities containing 'NaN'
        entities = [ent.text for ent in doc.ents if not any(char.isdigit() for char in ent.text) and 'nan' not in ent.text.lower()]

        # Counts the occurrences of each entity
        entity_counts = Counter(entities)

        # The 7 most common entities
        most_common_entities = entity_counts.most_common(7)

        txt_files_df.at[index, 'nlp_entities'] = most_common_entities
        txt_files_df = txt_files_df.rename(columns={"file_name": "file_path"})
    
    return txt_files_df

# FINAL FUNCTION 

In [43]:
def spreadsheet_maker(year):
    try:
        csv_files_df = nlp_entities_maker_csv(year)
        txt_files_df = nlp_entities_maker_txt(year)
        nlp_entities_year = pd.concat([txt_files_df, csv_files_df], axis=0)
        df_titles_year = title_maker(year)
        all_cols_year = pd.merge(df_titles_year, nlp_entities_year, on="file_path", how="left")
        return all_cols_year
    except Exception as e:
        print(f"Error in spreadsheet_maker: {e}")
        return None

In [44]:
metadata = spreadsheet_maker(1873)
metadata.to_csv(source_path + str(1873) + "/statistics_metadata.csv", index=False)

File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319736.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319905.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319948_b.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319783_b.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319878_12-13.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319656.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319916_a.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319796_8-9.csv
File name: /Users/katherinemika/

/var/folders/x5/8k4j_05j26vccc6wvb8y4cw00000gq/T/ipykernel_23224/141445153.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[0]['title_port']=df.loc[1]['title_port']


OSError: Cannot save file into a non-existent directory: '/Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/statistics'

## Testing & troubleshooting errors

`csv` files must be properly formatted for this script to work. If the encoding is funny, or there's column of values that are not aligned correctly, the spreadsheet maker will fail. It usually works to test whether a file can be read into a dataframe. If it can't then you have to manually inspect it, dpending on the error message, and go from there. 

In [9]:
folder_path_csv = source_path + str(1873) + '/csv'
files_list = os.listdir(folder_path_csv)
csv_files_df = pd.DataFrame(files_list, columns=['file_name'])

In [10]:
csv_files_df

,file_name
0,44319736.csv
1,44319905.csv
2,44319948_b.csv
3,44319783_b.csv
4,44319878_12-13.csv
...,...
341,44319649_a.csv
342,44319772_b.csv
343,44319713.csv
344,44319707.csv


In [12]:
for index, row in csv_files_df.iterrows():
    #print(index, row)
    print(folder_path_csv + '/' + str(row['file_name']))
    df = pd.read_csv(folder_path_csv + '/' + str(row["file_name"]))
    print(df)

/Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319736.csv
                                DESCRIPTION OF GOODS. CLASSIFIER OF QUANTITY.  \
0                                                 NaN                     NaN   
1                                                 NaN                     NaN   
2                              Silk, Coarse and Wild,                  Peculs   
3                                     „ Piece Goods,                        „   
4                       „ Manuftd., Goods, Unclassed,                       „   
5                                         Tea, Black,                       „   
6                                           Camphor,                        „   
7                                       Dates, Black,                       „   
8                           Dyes, Colours and Paints,                       „   
9                                Fish, of all kinds,                        „   
1

In [18]:
df = pd.read_csv("/Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319905.csv")
df

,Unnamed: 0,1863.,1864.,1865.,1866.,1867.,1868.,1869.,1870.,1871.,1872.
0,NaN,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c,Hk. Tts. m.c.c
1,"British,","1,817.9.2.3","24,333.7.2.1","22,135.5.3.8","11,724.2.3.3","23,545.4.9.5","16,915.4.3.8","17,377.7.3.2","30,045.0.5.4","52,714.5.7.8","58,395.8.6.8"
2,"American,",NaN,NaN,693.0.5.6,811.9.8.1,131.5.5.6,"1,423.1.9.7",NaN,"3,372.5.0.0","3,686.6.1.5","3,712.9.3.2"
3,"French,",NaN,694.0.6.6,"2,168.6.3.1",689.5.44,"3,293.5.6.3","3,918.0.7.1","3,152.8.1.4","3,937.7.8.0","6,972.3.8.4","3,311.8.8.2"
4,"Swedish and Norwegian,",NaN,"1,728.1.4.3","2,135.6.8.1",979.1.4.2,NaN,881.42.7,NaN,"5,953.6.0.8","7,995.9.2.0","3,766.8.2.1"
5,"Russian,",NaN,NaN,NaN,NaN,NaN,NaN,"2,052.0.0.0",NaN,662.6.7.6,NaN
6,"German,","6,818.6.3.0","11,523.7.8.4","21,367.0.54","34,574.9.3.9","40,324.5.8.5","27,558.1.3.3","43,008.7.4.5","49,000.0.6.1","21,682.1.9.2","33,014.9.3.2"
7,"Danish,","7,573.9.4.0","9,571.6.7.9","5,704.0.2.9",388.9.4.9,"1,175.9.8.9",NaN,"5,336.4.2.2","13,045.3.9.0","12,889.0.90","1,525.4.6.4"
8,"Dutch,",NaN,NaN,308.3.4.5,"2,166.4.4.7",NaN,790.6.9.0,"1,697.5.9.4","6,077.4.3.8","10,203.4.0.6","5,412.7.5.7"
9,"Spanish,",NaN,NaN,77.2.0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
test_csv = another_nlp_entities_maker_csv(1873)

File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319736.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319905.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319948_b.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319783_b.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319878_12-13.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319656.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319916_a.csv
File name: /Users/katherinemika/Desktop/curation/historic_datasets/trade_statistics_treaty_ports/1873/csv/44319796_8-9.csv
File name: /Users/katherinemika/